# Visualize attention

In [1]:
from torch.utils.data.dataloader import DataLoader
from sentiment_classification.reviewsDataset import reviewsDataset
from gpt_utils import dynamic_padding
import tiktoken
import torch
import torch.nn.functional as F
enc = tiktoken.get_encoding("gpt2")
def encode(s: str):
    return enc.encode(s, allowed_special={"<|endoftext|>"},)

rd = reviewsDataset(split="test")
batch_size = 1
dl = DataLoader(rd,shuffle=True,batch_size=batch_size,collate_fn=dynamic_padding)


In [2]:
from gpt import GPT
from gpt_config import GPTConfig
from bertviz import model_view,head_view

model_config = GPTConfig(use_lora=True,load_from_checkpoint=True,checkpoint_path="run/dropout_low_lr/finetune_lora.ckpt",debug=True)
# model_config = GPTConfig(debug=True,binary_classification_head=False)
model = GPT.from_pretrained(config=model_config)
ckpt = torch.load(model_config.checkpoint_path)
model.load_state_dict(ckpt["model"])
model.eval()
# device = "mps"
print("Loaded Model")

Loading pre-trained weights for gpt2
Number of parameters: 123.65M


/var/folders/p1/qmcgqwhj1mx925j0j_r36xp40000gn/T/ipykernel_91108/372774607.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_config.checkpoint_path

Loaded Model


In [5]:
torch.manual_seed(1339)
batch = next(iter(dl))
logits, _,att_out = model(batch["input_ids"],batch["attention_masks"])    
predictions = F.sigmoid(logits)
tokens = []

for input_id in batch["input_ids"][0]:
    tokens.append(f"{enc.decode([input_id.item()])}")

model_view(attention=att_out, tokens=tokens,include_layers=[11]) 

<IPython.core.display.Javascript object>